In [1]:
import os

train_data_path = "./data/train_essays.csv"
supplement_data_dir = "./data/archive/"
supplement_data_files = [  os.path.join(supplement_data_dir,f)  
                          for f in os.listdir(supplement_data_dir)
                          if(f.endswith('.csv'))]

In [2]:
def set_proxy():
    import os
    cache_dir = "/home/tx/workspace/cache"  # 替换为你想要的缓存目录的路径
    os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # 代理
    os.environ['http_proxy'] = 'http://127.0.0.1:7890'
    os.environ['https_proxy'] = 'http://127.0.0.1:7890'
    os.environ['no_proxy'] = '127.0.0.1,localhost'
    os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['NO_PROXY'] = '127.0.0.1,localhost'
    return
set_proxy()

In [3]:
!curl huggingface.co

<html>
<head><title>301 Moved Permanently</title></head>
<body>
<center><h1>301 Moved Permanently</h1></center>
<hr><center>CloudFront</center>
</body>
</html>


In [4]:
import pandas as pd

supplement_train_data = pd.concat([
    pd.read_csv(f)
    for f in supplement_data_files
])
train_data=pd.read_csv(train_data_path)

In [5]:
supplement_train_data

,text,label,source,fold,essay_id,prompt
0,There are alot reasons to keep our the despise...,0,persuade_corpus,2,NaN,NaN
1,Driving smart cars that drive by themself has ...,0,persuade_corpus,4,NaN,NaN
2,"Dear Principal,\n\nI believe that students at ...",0,persuade_corpus,0,NaN,NaN
3,"Dear Principal,\n\nCommunity service should no...",0,persuade_corpus,0,NaN,NaN
4,My argument for the development of the driverl...,0,persuade_corpus,3,NaN,NaN
...,...,...,...,...,...,...
39780,"""Oh man I didn't make the soccer team!"", yelle...",0,persuade_corpus,4,F7341069C4A4,NaN
39781,I believe that using this technology could be ...,0,persuade_corpus,2,AFE6E553DAC2,NaN
39782,The Face on Mars is a fascinating phenomenon t...,1,falcon_180b_v1,2,falcon_180b_v1_600,You have read the article 'Unmasking the Face ...
39783,Texting & Driving\n\nUsing your phone while dr...,0,persuade_corpus,2,A5F84C104693,NaN


In [6]:
train_data.columns

Index(['id', 'prompt_id', 'text', 'generated'], dtype='object')

In [7]:
import gc
#del train_data_re
gc.collect()
train_data_re = pd.DataFrame(train_data[['text','generated','']])


In [8]:
train_data

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


## build base_model

In [9]:
cache_dir = "/home/tx/workspace/cache"  # 替换为你想要的缓存目录的路径
from transformers import AutoModelForCausalLM,AutoTokenizer
from transformers import AutoConfig, LlamaConfig 
# from llava.model.language_model.llava_llama import LlavaLlamaForCausalLM
# class LlavaConfig(LlamaConfig):
#     model_type = "llava"
# AutoConfig.register("llava", LlavaConfig)
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
#model_name = "ChocoWu/nextgpt_7b_tiva_v0"#"liuhaotian/llava-v1.5-7b"
model = AutoModelForCausalLM.from_pretrained(model_name,cache_dir=cache_dir,trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
# tokenizer.add_tokens(["[PAD]"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## dataset

In [18]:
#train_data_re

In [11]:
def generate_prompt(prompt,text):
    """
    
    """
    text= None 
    if(prompt is not None):
        template_with_prompt = f"""
        a prompt is shown as follows: 
        {prompt}.
        text generated by this this prompt is shown as below:
        {text}.
        is this text generated by AI?
        """
        text = template_with_prompt
    else:
        template_without_prompt = f"""
        text generated without any prompt is shown as below:
        {text}.
        is this text generated by AI?
        """
        text = template_without_prompt



    return text

2

In [20]:
from torch import Tensor
from torch.utils.data import TensorDataset,DataLoader,RandomSampler

class myDataset(TensorDataset):

    def __init__(self, datalist,max_length=256,tokenizer=None,preprocess_func= None) -> None:
        super(myDataset,self).__init__()

        if(isinstance(datalist,pd.DataFrame)):
            self.datalist = datalist.to_dict(orient='list')
        elif(isinstance(datalist,dict)):
            self.datalist = datalist
        else:
            raise Exception("错误输入类型")
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.preprocess_func = preprocess_func

    def __len__(self):
        return len(self.datalist)
    
    def __getitem__(self, index):
        text = self.datalist['text'][index]  
        prompt = self.datalist['prompt'][index]  

        final_text = self.preprocess_func(prompt,text)
        input_ids = self.tokenizer.encode(final_text)
        att_mask = [1] * len(input_ids)

        if('generated' in self.datalist):
            generated = self.datalist['generated'][index]
            label_text = "the text is generated by AI" if generated > 0 else "the text is written by students"
            label_ids = self.tokenizer.encode(label_text)

            input_ids = input_ids + label_ids
            att_mask = [1] * len(input_ids)
        while(len(input_ids) < self.max_length):
            input_ids.append(self.tokenizer.eos_token_id)
            att_mask.append(0)
            
        return {'input_ids':input_ids,'att_mask':att_mask}

        
dataset = myDataset(datalist=train_data_re,max_length=1024,tokenizer=tokenizer) 




In [21]:
dataset[0]

KeyError: 'prompt'

In [14]:
random_sampler = RandomSampler(dataset)

In [15]:

dataloader_random = DataLoader(dataset, batch_size=2, sampler=random_sampler)


In [16]:
import torch

In [17]:
model(input_ids = torch.LongTensor([dataset[0]['input_ids']]),attention_mask=torch.LongTensor([dataset[0]['att_mask']]))

CausalLMOutputWithPast(loss=None, logits=tensor([[[-4.6542, -4.4159, -0.0891,  ..., -3.4539, -2.4985, -3.0392],
         [-8.4642, -8.6666,  2.4634,  ..., -7.9056, -6.1161, -3.8112],
         [-8.1824, -8.3283,  2.0552,  ..., -5.7254, -4.7424, -1.6529],
         ...,
         [-6.8368, -1.3740, 11.2287,  ..., -6.4220, -5.9970, -4.2193],
         [-6.7842, -1.2774, 11.1945,  ..., -6.4234, -5.9681, -4.2548],
         [-6.7887, -1.2739, 11.2149,  ..., -6.4319, -6.0761, -4.3294]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-4.3710e-01,  1.9023e-01,  7.2229e-01,  ..., -1.6132e+00,
           -2.1386e+00, -2.2042e+00],
          [ 6.1248e+00, -4.8535e+00, -2.2835e+00,  ...,  2.1415e+00,
            1.2255e+00,  5.5861e-01],
          [ 1.1742e+00, -1.6320e+00,  7.6247e-01,  ...,  1.6744e+00,
            1.4452e+00,  6.2961e-01],
          ...,
          [-1.1546e+00,  1.5177e+00, -9.8412e-01,  ...,  4.6892e-01,
            2.6624e-01,  2.5282e-01],
          [-1.839

## model definition

In [ ]:
import torch 

class ai_classifier(torch.nn.Module):

    def __init__(self, base_model) -> None:
        super(ai_classifier,self).__init__()

        self.base_model = base_model

    def forward(self,inputs):
        self.base_model()

    def forward(self,input,tokenizer):
        pass



In [1]:
model

NameError: name 'model' is not defined